<a href="https://colab.research.google.com/github/Lucky-Peter-Okonun/Advance-Practical-course-Data-Science/blob/main/Task_2_Adv_Dat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 ! git clone https://github.com/Lucky-Peter-Okonun/DNNTSP.git


Cloning into 'DNNTSP'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 76 (delta 30), reused 64 (delta 19), pack-reused 0
Unpacking objects: 100% (76/76), done.


In [ ]:
ls

DNNTSP/  sample_data/


In [ ]:
! rm -r sample_data
!mv /content/DNNTSP/*   /content/
!rm -r DNNTSP

In [ ]:
!ls

data  model  README.md	results  runs  save_model_folder  test	train  utils


In [ ]:
cat utils/config.json

{
    "data": "TaFeng",
    "save_model_folder": "DNNTSP",

    "item_embed_dim": 32,

    "cuda": 0,
    "loss_function": "multi_label_soft_loss",
    "epochs": 100,
    "batch_size": 64,
    "learning_rate": 0.001,
    "optim": "Adam",
    "weight_decay": 0
}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/DNNTSP-master/')

In [ ]:
# cat gdrive/MyDrive/DNNTSP/utils/config.json

cat: gdrive/MyDrive/DNNTSP/utils/config.json: No such file or directory


In [ ]:
cat utils/config.json

{
    "data": "TaFeng",
    "save_model_folder": "DNNTSP",

    "item_embed_dim": 32,

    "cuda": 0,
    "loss_function": "multi_label_soft_loss",
    "epochs": 100,
    "batch_size": 64,
    "learning_rate": 0.001,
    "optim": "Adam",
    "weight_decay": 0
}

In [ ]:
! cat  README.md

# Predicting Temporal Sets with Deep Neural Networks (DNNTSP)

DNNTSP is a general neural network architecture that could make prediction on temporal sets.

Please refer to our KDD 2020 paper [“**Predicting Temporal Sets with Deep Neural Networks**”](https://dl.acm.org/doi/pdf/10.1145/3394486.3403152) for more details.

## Project Structure
The descriptions of principal files in this project are explained as follows:
- ./model/
    - `weighted_graph_conv.py`: codes for the Element Relationship Learning component (i.e. weighted GCN on dynamic graphs)
    - `masked_self_attention.py` and `aggregate_nodes_temporal_feature.py`: codes for the Attention-based Temporal Dependency Learning component (i.e. masked self-attention and weighted aggregation of temporal information)
    - `global_gated_update.py`: codes for the Gated Information Fusing component (i.e. gated updating mechanism)
- ./train/
  - `train_model.py` and `train_main.py`: codes for training models
- ./test/
  - `testing_model.

In [ ]:
pip install torch==1.5.0+cu101 torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 7.6MB 4.3MB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.5.0+cu101 which is incompatible.
ERROR: torchaudio 0.7.2 has requirement torch==1.7.1, but you'll have torch 1.5.0+cu101 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [ ]:
pip install  tqdm numpy pandas sklearn tensorboardX  dgl-cu101

     |████████████████████████████████| 317kB 5.4MB/s 
     |████████████████████████████████| 25.0MB 43.0MB/s 


In [ ]:
!ls train

train_main.py  train_model.py


In [ ]:
! cat train/train_main.py

import torch
import torch.nn as nn
import sys
import os
import shutil

from utils.load_config import get_attribute
from utils.data_container import get_data_loader
from utils.loss import BPRLoss, WeightMSELoss
from utils.util import get_class_weights

from model.temporal_set_prediction import temporal_set_prediction
from train.train_model import train_model


def create_model():

    print(f"{get_attribute('data')}/{get_attribute('save_model_folder')}")

    model = temporal_set_prediction(items_total=get_attribute('items_total'),
                                    item_embedding_dim=get_attribute('item_embed_dim'))

    return model


def create_loss(loss_type):
    if loss_type == 'bpr_loss':
        loss_func = BPRLoss()
    elif loss_type == 'mse_loss':
        loss_func = WeightMSELoss()
    elif loss_type == 'weight_mse_loss':
        loss_func = WeightMSELoss(weights=get_class_weights(get_attribute('data_path')))
    elif loss_type == "multi_label_soft_loss":
        loss_func 

In [ ]:
!ls /content/gdrive/MyDrive/DNNTSP-master

data  model  README.md	results  runs  save_model_folder  test	train  utils


In [ ]:
import torch
import torch.nn as nn
import sys
import os
import shutil

from utils.load_config import get_attribute
from utils.data_container import get_data_loader
from utils.loss import BPRLoss, WeightMSELoss
from utils.util import get_class_weights

from model.temporal_set_prediction import temporal_set_prediction
from train.train_model import train_model


def create_model():

    print(f"{get_attribute('data')}/{get_attribute('save_model_folder')}")

    model = temporal_set_prediction(items_total=get_attribute('items_total'),
                                    item_embedding_dim=get_attribute('item_embed_dim'))

    return model


def create_loss(loss_type):
    if loss_type == 'bpr_loss':
        loss_func = BPRLoss()
    elif loss_type == 'mse_loss':
        loss_func = WeightMSELoss()
    elif loss_type == 'weight_mse_loss':
        loss_func = WeightMSELoss(weights=get_class_weights(get_attribute('data_path')))
    elif loss_type == "multi_label_soft_loss":
        loss_func = nn.MultiLabelSoftMarginLoss(reduction="mean")
    else:
        raise ValueError("Unknown loss function.")
    return loss_func


def train():
    model = create_model()
    # 创建data_loader
    train_data_loader = get_data_loader(data_path=get_attribute('data_path'),
                                        data_type='train',
                                        batch_size=get_attribute('batch_size'),
                                        item_embedding_matrix=model.item_embedding)
    valid_data_loader = get_data_loader(data_path=get_attribute('data_path'),
                                        data_type='validate',
                                        batch_size=get_attribute('batch_size'),
                                        item_embedding_matrix=model.item_embedding)
    loss_func = create_loss(loss_type=get_attribute('loss_function'))

    # 训练
    model_folder = f"../save_model_folder/{get_attribute('data')}/{get_attribute('save_model_folder')}"
    tensorboard_folder = f"../runs/{get_attribute('data')}/{get_attribute('save_model_folder')}"

    shutil.rmtree(model_folder, ignore_errors=True)
    os.makedirs(model_folder, exist_ok=True)
    shutil.rmtree(tensorboard_folder, ignore_errors=True)
    os.makedirs(tensorboard_folder, exist_ok=True)

    if get_attribute("optim") == "Adam":
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=get_attribute("learning_rate"),
                                     weight_decay=get_attribute("weight_decay"))
    elif get_attribute("optim") == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=get_attribute("learning_rate"),
                                    momentum=0.9)
    else:
        raise NotImplementedError()

    train_model(model=model,
                train_data_loader=train_data_loader,
                valid_data_loader=valid_data_loader,
                loss_func=loss_func,
                epochs=get_attribute('epochs'),
                optimizer=optimizer,
                model_folder=model_folder,
                tensorboard_folder=tensorboard_folder)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
train()

TaFeng/DNNTSP
train data length -> 6888


  0%|          | 0/108 [00:00<?, ?it/s]

validate data length -> 984
temporal_set_prediction(
  (item_embedding): Embedding(4935, 32)
  (stacked_gcn): stacked_weighted_GCN_blocks(
    (0): weighted_GCN(
      (gcns): ModuleList(
        (0): weighted_graph_conv(
          (linear): Linear(in_features=32, out_features=32, bias=True)
        )
        (1): weighted_graph_conv(
          (linear): Linear(in_features=32, out_features=32, bias=True)
        )
      )
      (relus): ModuleList(
        (0): ReLU()
        (1): ReLU()
      )
      (bns): ModuleList(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (masked_self_attention): masked_self_attention(
    (Wq): Linear(in_features=32, out_features=32, bias=False)
    (Wk): Linear(in_features=32, out_features=32, bias=False)
    (Wv): Linear(in_features=32, out_features=32, bias=False)
  )
  (aggregate_nodes_temporal_fe

train epoch: 0, train loss: 0.7029649046836076: 100%|██████████| 108/108 [02:34<00:00,  1.43s/it]


train metric ...


...

train metric ...


  0%|          | 0/16 [00:00<?, ?it/s]

{
    "PHR_10": 0.562862950058072,
    "PHR_20": 0.6443089430894309,
    "PHR_30": 0.6853948896631823,
    "PHR_40": 0.7106562137049942,
    "ndcg_10": 0.20943857729434967,
    "ndcg_20": 0.22663076221942902,
    "ndcg_30": 0.23727838695049286,
    "ndcg_40": 0.24482594430446625,
    "recall_10": 0.22915621101856232,
    "recall_20": 0.29058578610420227,
    "recall_30": 0.32442182302474976,
    "recall_40": 0.34928926825523376
}


validate epoch: 99, validate loss: 0.0077771150099579245: 100%|██████████| 16/16 [00:12<00:00,  1.33it/s]


validate metric ...
{
    "PHR_10": 0.44715447154471544,
    "PHR_20": 0.5640243902439024,
    "PHR_30": 0.6148373983739838,
    "PHR_40": 0.657520325203252,
    "ndcg_10": 0.1393987387418747,
    "ndcg_20": 0.16048966348171234,
    "ndcg_30": 0.17169684171676636,
    "ndcg_40": 0.18014846742153168,
    "recall_10": 0.1616867482662201,
    "recall_20": 0.22584471106529236,
    "recall_30": 0.26045796275138855,
    "recall_40": 0.2878376841545105
}
cost 17415 seconds


In [ ]:
ls

data/    model/      results/  save_model_folder/  train/
gdrive/  README.md*  runs/     test/               utils/


In [ ]:
cat test/testing_model.py

import json
import sys
import os

from utils.metric import evaluate
from utils.data_container import get_data_loader
from utils.load_config import get_attribute
from utils.util import convert_to_gpu
from train.train_main import create_model
from utils.util import load_model


if __name__ == '__main__':
    model_path = f"../save_model_folder/{get_attribute('data')}/{get_attribute('save_model_folder')}" \
        f"/model_epoch_19.pkl"
    print(f'model path -> {model_path}')

    model = create_model()
    model = load_model(model, model_path)
    model = convert_to_gpu(model)
    print(model)

    test_data_loader = get_data_loader(data_path=get_attribute('data_path'),
                                       data_type='test',
                                       batch_size=get_attribute('batch_size'),
                                       item_embedding_matrix=model.item_embedding)

    print('===== Test predict result =====')
    scores = evaluate(model,
                      test_

In [ ]:
import json
import sys
import os

from utils.metric import evaluate
from utils.data_container import get_data_loader
from utils.load_config import get_attribute
from utils.util import convert_to_gpu
from train.train_main import create_model
from utils.util import load_model


if __name__ == '__main__':
    model_path = "/content/gdrive/MyDrive/DNNTSP-master/save_model_folder/model_epoch_1.pkl"  
  
    print(f'model path -> {model_path}')

    model = create_model()
    model = load_model(model, model_path)
    model = convert_to_gpu(model)
    print(model)

    test_data_loader = get_data_loader(data_path=get_attribute('data_path'),
                                       data_type='test',
                                       batch_size=get_attribute('batch_size'),
                                       item_embedding_matrix=model.item_embedding)

    print('===== Test predict result =====')
    scores = evaluate(model,
                      test_data_loader)

    scores = sorted(scores.items(), key=lambda item: item[0], reverse=False)
    scores = {item[0]: item[1] for item in scores}

    scores_str = json.dumps(scores, indent=4)
    print(f'scores -> {scores_str}')

    model_folder = f"../results/{get_attribute('data')}"
    if not os.path.exists(model_folder):
        os.makedirs(model_folder, exist_ok=True)

    #save_path = f"{model_folder}/{get_attribute('save_model_folder')}_result.json"
    #print(f'save path is {save_path}')

    with open("results.json", 'w') as file:
        file.write(scores_str)
 

model path -> /content/gdrive/MyDrive/DNNTSP-master/save_model_folder/model_epoch_1.pkl
TaFeng/DNNTSP
temporal_set_prediction(
  (item_embedding): Embedding(4935, 32)
  (stacked_gcn): stacked_weighted_GCN_blocks(
    (0): weighted_GCN(
      (gcns): ModuleList(
        (0): weighted_graph_conv(
          (linear): Linear(in_features=32, out_features=32, bias=True)
        )
        (1): weighted_graph_conv(
          (linear): Linear(in_features=32, out_features=32, bias=True)
        )
      )
      (relus): ModuleList(
        (0): ReLU()
        (1): ReLU()
      )
      (bns): ModuleList(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (masked_self_attention): masked_self_attention(
    (Wq): Linear(in_features=32, out_features=32, bias=False)
    (Wk): Linear(in_features=32, out_features=32, bias=False)
    (Wv): Linear(in_fe

  0%|          | 0/31 [00:00<?, ?it/s]

test data length -> 1969
===== Test predict result =====


100%|██████████| 31/31 [00:22<00:00,  1.38it/s]


scores -> {
    "PHR_10": 0.015236160487557136,
    "PHR_20": 0.02437785678009142,
    "PHR_30": 0.029964448958862366,
    "PHR_40": 0.041137633316404264,
    "ndcg_10": 0.002009330317378044,
    "ndcg_20": 0.0025547598488628864,
    "ndcg_30": 0.0029573296196758747,
    "ndcg_40": 0.0035013165324926376,
    "recall_10": 0.0031192232854664326,
    "recall_20": 0.004659412894397974,
    "recall_30": 0.006085589528083801,
    "recall_40": 0.0075263530015945435
}


In [ ]:
ls

data/    model/      results/      runs/               test/   utils/
gdrive/  README.md*  results.json  save_model_folder/  train/


# New Section

In [ ]:
 ls /content/gdrive/MyDrive/DNNTSP-master/save_model_folder/

model_epoch_0.pkl  model_epoch_1.pkl


In [ ]:
ls

data/    model/      results/  save_model_folder/  train/
gdrive/  README.md*  runs/     test/               utils/


In [ ]:
cat results.json

{
    "PHR_10": 0.015236160487557136,
    "PHR_20": 0.02437785678009142,
    "PHR_30": 0.029964448958862366,
    "PHR_40": 0.041137633316404264,
    "ndcg_10": 0.002009330317378044,
    "ndcg_20": 0.0025547598488628864,
    "ndcg_30": 0.0029573296196758747,
    "ndcg_40": 0.0035013165324926376,
    "recall_10": 0.0031192232854664326,
    "recall_20": 0.004659412894397974,
    "recall_30": 0.006085589528083801,
    "recall_40": 0.0075263530015945435
}